# Applied Data Science Capstone
## Project Title: The Battle of Neighborhoods

In [4]:
#import libraries
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
C_ID = 'NSNZ2FAIXDEOCGAZGQ2LBMBV3YMQL1X0J1KHFE1UA40R0SEB'
C_SC = 'KFLH2TZRLVJXWMOLWVBDEGPBUFHS3OMGGH5BQRVV2PG5EECS'
V = 20200509
L = 100
R = 500

In [3]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(C_ID,C_SC,V,'22.302711','114.177216',R,L)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=NSNZ2FAIXDEOCGAZGQ2LBMBV3YMQL1X0J1KHFE1UA40R0SEB&client_secret=KFLH2TZRLVJXWMOLWVBDEGPBUFHS3OMGGH5BQRVV2PG5EECS&v=20200509&ll=22.302711,114.177216&radius=500&limit=100'

In [11]:
results = requests.get(url).json()
results

: 114.17962696655532}],
        'distance': 360,
        'cc': 'HK',
        'city': '香港',
        'country': '香港',
        'formattedAddress': ['2/F, Hotel ICON, 17 Science Museum Rd', '香港']},
       'categories': [{'id': '52e81612bcbc57f1066b79f4',
         'name': 'Buffet',
         'pluralName': 'Buffets',
         'shortName': 'Buffet',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []},
       'venuePage': {'id': '60362304'}},
      'referralId': 'e-0-4e93e5ce77c82b6a71c1341c-14'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b0cdf33f964a520524223e3',
       'name': 'Kowloon Cricket Club (九龍木球會)',
       'location': {'address': "10 Cox's Rd",
        'lat': 22.304594,
        'lng': 114.174218,
        'labe

In [8]:
def getCategoryType(row):
    try:
        cate_list = row['categories']
    except:
        cate_list = row['venue.categories']
    if len(cate_list) == 0:
        return None
    else:
        return cate_list[0]['name'] 

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name','venue.location.city', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(getCategoryType, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.shape

(32, 5)

In [24]:
#get a list of neighborhoods on SH
#get the wiki HTML and read it in soup
from bs4 import BeautifulSoup
import urllib
url = 'https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Shanghai'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html,'html.parser')

In [35]:
#find targeted class of content
table = soup.find('div',{'class':'mw-content-ltr'})
#find all links
allLinks = table.findAll('a')
#expell all useless link
all_nei_links = allLinks[13:]
all_nei_links
#store links title to a list
sh_nei = []
for link in all_nei_links:
    sh_nei.append(link.get('title'))
#done
#sh_nei
df_sh = pd.DataFrame()
df_sh['Neighborhood'] = sh_nei
df_sh

,Neighborhood
0,Anting
1,Changshou Road Subdistrict
2,Fengjing
3,"Gaoqiao, Shanghai"
4,"Gubei, Shanghai"
5,"Koreatown, Shanghai"
6,Lujiazui
7,"Luodian, Shanghai"
8,Nanxiang
9,Qiantan International Business Zone (Shanghai)


In [ ]:
#get neighborhood coordinates and append to df_sh, if no result, then return nan
def getCoord()
